# Physics-Informed Neural Network (PINN) for Granular Segregation - Phase 2.1

This notebook implements a PINN for inverse modeling of the non-dimensionalized segregation PDE.

## Phase 2.1: Learnable Segregation Parameter Λ

In this phase, we learn the segregation parameter $\Lambda$ as a **learnable parameter** during training. The functional form of the segregation velocity $\Lambda(1-\tilde{x})g(\tilde{z})(1-c)$ is maintained, but instead of using a fixed value for $\Lambda$, it becomes a trainable parameter that is optimized along with the neural network weights.

## Non-dimensionalized PDE

The governing equation is:

$$\frac{\partial c}{\partial \tilde{t}} + \tilde{u} \frac{\partial c}{\partial \tilde{x}} + \tilde{w} \frac{\partial c}{\partial \tilde{z}} + \Lambda(1 - \tilde{x}) \frac{\partial}{\partial \tilde{z}} \left[ g(\tilde{z}) c(1-c) \right] = \frac{\partial}{\partial \tilde{z}} \left[ \frac{1}{Pe} \frac{\partial c}{\partial \tilde{z}} \right]$$

Expanding the segregation term:

$$\Lambda(1-\tilde{x}) \frac{\partial}{\partial \tilde{z}} \left[ g(\tilde{z}) c(1-c) \right] = \Lambda(1-\tilde{x}) \left[ g'(\tilde{z}) c(1-c) + g(\tilde{z})(1-2c) \frac{\partial c}{\partial \tilde{z}} \right]$$

Where:
- $c$ is the concentration (volume fraction of small particles)
- $\tilde{x} \in [0, 1]$, $\tilde{z} \in [-1, 0]$, $\tilde{t} \in [0, t_{end}]$ are dimensionless coordinates
- $\Lambda$ is the **learnable segregation parameter** (optimized during training)
- $Pe$ is the Péclet number
- $\tilde{u}$ and $\tilde{w}$ are dimensionless velocity profiles
- $g(\tilde{z})$ is a shear-rate-like profile

## Boundary Conditions

- **Inlet** ($\tilde{x} = 0$): Dirichlet $c = 0.5$ (well-mixed feed)
- **Top/Bottom** ($\tilde{z} = 0, -1$): Neumann $(1/Pe) \partial c/\partial \tilde{z} = \Lambda(1-\tilde{x}) g(\tilde{z}) c(1-c)$
- **Outlet** ($\tilde{x} = 1$): Natural boundary condition (zero diffusive flux)
- **Initial condition**: $c = 0.5$ everywhere at $\tilde{t} = 0$


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)


In [ ]:
# Parameters from the paper (section 3)
# Lambda will be a learnable parameter (defined in the PINN class)
Lambda_init = 0.3949  # Initial guess for Λ (will be learned)
Pe = 4.0           # Péclet number
D = 1.0 / Pe        # vertical diffusivity
k = 2.3             # exponential velocity profile parameter
tEnd = 20.0         # final dimensionless time

# Domain bounds
x_min, x_max = 0.0, 1.0    # x̃ ∈ [0, 1]
z_min, z_max = -1.0, 0.0   # z̃ ∈ [-1, 0]
t_min, t_max = 0.0, tEnd   # t̃ ∈ [0, tEnd]

print(f"Parameters: Λ_init={Lambda_init}, Pe={Pe}, D={D:.6f}, k={k}")
print(f"Domain: x̃ ∈ [{x_min}, {x_max}], z̃ ∈ [{z_min}, {z_max}], t̃ ∈ [{t_min}, {t_max}]")


In [ ]:
# Helper functions for velocity profiles and g(z̃)
def u_tilde(x, z, k):
    """Dimensionless velocity in x direction"""
    # Convert k to tensor if it's not already a tensor
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=x.dtype, device=x.device)
    factor = 0.5 * k * (1 - torch.exp(-k))
    return factor * (1 - x) * torch.exp(k * z)

def w_tilde(z, k):
    """Dimensionless velocity in z direction"""
    # Convert k to tensor if it's not already a tensor
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=z.dtype, device=z.device)
    factor = 0.5 * (1 - torch.exp(-k))
    return factor * (torch.exp(k * z) - 1)

def g_profile(z, k):
    """Shear-rate-like profile g(z̃) and its derivative"""
    # Convert k to tensor if it's not already a tensor
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=z.dtype, device=z.device)
    g_val = (k**2 / (2 * (1 - torch.exp(-k)))) * torch.exp(k * z)
    g_prime = k * g_val
    return g_val, g_prime


In [ ]:
# Neural Network Architecture with Learnable Lambda
class PINN(nn.Module):
    def __init__(self, layers, lambda_init=0.78):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
        
        # Initialize weights using Xavier initialization
        for layer in self.layers:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.zeros_(layer.bias)
        
        # Lambda as a learnable parameter
        # Use log-space to ensure Lambda stays positive during training
        self.log_lambda = nn.Parameter(torch.tensor(np.log(lambda_init), dtype=torch.float32))
    
    @property
    def Lambda(self):
        """Get Lambda value (always positive due to exp)"""
        return torch.exp(self.log_lambda)
    
    def forward(self, x, z, t):
        # Concatenate inputs: [x, z, t]
        inputs = torch.cat([x, z, t], dim=1)
        
        # Forward pass through hidden layers
        for i, layer in enumerate(self.layers[:-1]):
            inputs = torch.tanh(layer(inputs))
        
        # Output layer (no activation for regression)
        # output = self.layers[-1](inputs)
        
        # Output layer with sigmoid activation to bound concentration between 0 and 1
        output = torch.sigmoid(self.layers[-1](inputs))
        return output

# Network architecture: 3 inputs (x, z, t) -> hidden layers -> 1 output (c)
layers = [3, 64, 64, 64, 64, 1]
model = PINN(layers, lambda_init=Lambda_init).to(device)
print(f"Model architecture: {layers}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Initial Lambda: {model.Lambda.item():.6f}")


In [ ]:
def compute_pde_residual(x, z, t, model, Pe, k):
    """
    Compute the PDE residual:
    ∂c/∂t̃ = (1/Pe) ∂²c/∂z̃² - ũ ∂c/∂x̃ - w̃ ∂c/∂z̃ 
            - Λ(1-x̃)[ g'(z̃)c(1-c) + g(z̃)(1-2c)∂c/∂z̃ ]
    Note: Lambda is now a learnable parameter accessed via model.Lambda
    """
    # Clone and detach tensors to avoid modifying originals, then enable gradient computation
    x = x.clone().detach().requires_grad_(True)
    z = z.clone().detach().requires_grad_(True)
    t = t.clone().detach().requires_grad_(True)
    
    # Forward pass
    c = model(x, z, t)
    
    # Compute gradients
    c_t = grad(c, t, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    c_x = grad(c, x, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    c_z = grad(c, z, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    
    # Second derivative in z
    c_zz = grad(c_z, z, grad_outputs=torch.ones_like(c_z), create_graph=True)[0]
    
    # Velocity profiles
    u_tilde_val = u_tilde(x, z, k)
    w_tilde_val = w_tilde(z, k)
    
    # g profile and its derivative
    g_val, g_prime = g_profile(z, k)
    
    # Get learnable Lambda from model
    Lambda = model.Lambda
    
    # Diffusion term
    diff_term = (1.0 / Pe) * c_zz
    
    # Advection term
    adv_term = u_tilde_val * c_x + w_tilde_val * c_z
    
    # Segregation term (using learnable Lambda)
    seg_term = Lambda * (1 - x) * (g_prime * c * (1 - c) + g_val * (1 - 2*c) * c_z)
    
    # PDE residual: ∂c/∂t - (diffusion - advection - segregation)
    residual = c_t - (diff_term - adv_term - seg_term)
    
    return residual, c


In [ ]:
# Generate training points
def generate_training_data(n_pde, n_bc_inlet, n_bc_top, n_bc_bottom, n_bc_outlet, n_ic):
    """
    Generate collocation points for:
    - PDE residual points (interior)
    - Boundary condition points
    - Initial condition points
    """
    # PDE collocation points (interior domain)
    x_pde = torch.rand(n_pde, 1, device=device) * (x_max - x_min) + x_min
    z_pde = torch.rand(n_pde, 1, device=device) * (z_max - z_min) + z_min
    t_pde = torch.rand(n_pde, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: x̃ = 0 (inlet, Dirichlet: c = 0.5)
    x_bc_inlet = torch.zeros(n_bc_inlet, 1, device=device)
    z_bc_inlet = torch.rand(n_bc_inlet, 1, device=device) * (z_max - z_min) + z_min
    t_bc_inlet = torch.rand(n_bc_inlet, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: z̃ = 0 (top, Neumann)
    x_bc_top = torch.rand(n_bc_top, 1, device=device) * (x_max - x_min) + x_min
    z_bc_top = torch.zeros(n_bc_top, 1, device=device)
    t_bc_top = torch.rand(n_bc_top, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: z̃ = -1 (bottom, Neumann)
    x_bc_bottom = torch.rand(n_bc_bottom, 1, device=device) * (x_max - x_min) + x_min
    z_bc_bottom = torch.ones(n_bc_bottom, 1, device=device) * z_min
    t_bc_bottom = torch.rand(n_bc_bottom, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: x̃ = 1 (outlet, zero diffusive flux - natural BC, handled by PDE)
    x_bc_outlet = torch.ones(n_bc_outlet, 1, device=device)
    z_bc_outlet = torch.rand(n_bc_outlet, 1, device=device) * (z_max - z_min) + z_min
    t_bc_outlet = torch.rand(n_bc_outlet, 1, device=device) * (t_max - t_min) + t_min
    
    # Initial condition: t̃ = 0, c = 0.5 everywhere
    x_ic = torch.rand(n_ic, 1, device=device) * (x_max - x_min) + x_min
    z_ic = torch.rand(n_ic, 1, device=device) * (z_max - z_min) + z_min
    t_ic = torch.zeros(n_ic, 1, device=device)
    
    return {
        'pde': (x_pde, z_pde, t_pde),
        'bc_inlet': (x_bc_inlet, z_bc_inlet, t_bc_inlet),
        'bc_top': (x_bc_top, z_bc_top, t_bc_top),
        'bc_bottom': (x_bc_bottom, z_bc_bottom, t_bc_bottom),
        'bc_outlet': (x_bc_outlet, z_bc_outlet, t_bc_outlet),
        'ic': (x_ic, z_ic, t_ic)
    }

# Number of training points
n_pde = 10000
n_bc_inlet = 1000
n_bc_top = 1000
n_bc_bottom = 1000
n_bc_outlet = 1000
n_ic = 2000

train_data = generate_training_data(n_pde, n_bc_inlet, n_bc_top, n_bc_bottom, n_bc_outlet, n_ic)
print("Training data generated:")
for key, (x, z, t) in train_data.items():
    print(f"  {key}: {x.shape[0]} points")


In [ ]:
def compute_loss(model, train_data, Pe, k, weights, exp_data=None):
    """
    Compute total loss:
    L = w_pde * L_pde + w_bc_inlet * L_bc_inlet + w_bc_top * L_bc_top 
        + w_bc_bottom * L_bc_bottom + w_ic * L_ic + w_data * L_data
    
    Args:
        model: PINN model (contains learnable Lambda)
        train_data: Dictionary of training collocation points
        Pe: Péclet number
        k: Velocity profile parameter
        weights: Dictionary of loss weights
        exp_data: Optional experimental data dictionary with keys:
            'x', 'z', 't', 'c' (concentration measurements)
    """
    # Get learnable Lambda from model
    Lambda = model.Lambda
    
    total_loss = 0.0
    
    # PDE residual loss
    x_pde, z_pde, t_pde = train_data['pde']
    residual, _ = compute_pde_residual(x_pde, z_pde, t_pde, model, Pe, k)
    loss_pde = torch.mean(residual**2)
    total_loss += weights['pde'] * loss_pde
    
    # Boundary condition: x̃ = 0, c = 0.5 (Dirichlet)
    x_in, z_in, t_in = train_data['bc_inlet']
    c_in = model(x_in, z_in, t_in)
    loss_bc_inlet = torch.mean((c_in - 0.5)**2)
    total_loss += weights['bc_inlet'] * loss_bc_inlet
    
    # Boundary condition: z̃ = 0 (top, Neumann: (1/Pe) ∂c/∂z = Λ(1-x) g(z) c(1-c))
    # Clone tensors to avoid modifying originals
    x_top, z_top, t_top = train_data['bc_top']
    x_top = x_top.clone().detach().requires_grad_(True)
    z_top = z_top.clone().detach().requires_grad_(True)
    t_top = t_top.clone().detach()
    c_top = model(x_top, z_top, t_top)
    c_z_top = grad(c_top, z_top, grad_outputs=torch.ones_like(c_top), create_graph=True)[0]
    g_val_top, _ = g_profile(z_top, k)
    bc_top_target = Lambda * (1 - x_top) * g_val_top * c_top * (1 - c_top)
    bc_top_pred = (1.0 / Pe) * c_z_top
    loss_bc_top = torch.mean((bc_top_pred - bc_top_target)**2)
    total_loss += weights['bc_top'] * loss_bc_top
    
    # Boundary condition: z̃ = -1 (bottom, Neumann: (1/Pe) ∂c/∂z = Λ(1-x) g(z) c(1-c))
    # Clone tensors to avoid modifying originals
    x_bot, z_bot, t_bot = train_data['bc_bottom']
    x_bot = x_bot.clone().detach().requires_grad_(True)
    z_bot = z_bot.clone().detach().requires_grad_(True)
    t_bot = t_bot.clone().detach()
    c_bot = model(x_bot, z_bot, t_bot)
    c_z_bot = grad(c_bot, z_bot, grad_outputs=torch.ones_like(c_bot), create_graph=True)[0]
    g_val_bot, _ = g_profile(z_bot, k)
    bc_bot_target = Lambda * (1 - x_bot) * g_val_bot * c_bot * (1 - c_bot)
    bc_bot_pred = (1.0 / Pe) * c_z_bot
    loss_bc_bottom = torch.mean((bc_bot_pred - bc_bot_target)**2)
    total_loss += weights['bc_bottom'] * loss_bc_bottom
    
    # Initial condition: t̃ = 0, c = 0.5
    x_ic, z_ic, t_ic = train_data['ic']
    c_ic = model(x_ic, z_ic, t_ic)
    loss_ic = torch.mean((c_ic - 0.5)**2)
    total_loss += weights['ic'] * loss_ic
    
    # Experimental data loss (if provided)
    loss_data = torch.tensor(0.0, device=device)
    if exp_data is not None:
        x_data = exp_data['x']
        z_data = exp_data['z']
        t_data = exp_data['t']
        c_data = exp_data['c']  # Measured concentration values
        
        # Predict concentration at experimental points
        c_pred = model(x_data, z_data, t_data)
        loss_data = torch.mean((c_pred - c_data)**2)
        total_loss += weights.get('data', 1.0) * loss_data
    
    # Outlet boundary (x̃ = 1): zero diffusive flux is naturally satisfied
    # by the PDE, so we don't need to enforce it explicitly
    
    loss_dict = {
        'pde': loss_pde.item(),
        'bc_inlet': loss_bc_inlet.item(),
        'bc_top': loss_bc_top.item(),
        'bc_bottom': loss_bc_bottom.item(),
        'ic': loss_ic.item(),
        'lambda': Lambda.item()
    }
    
    if exp_data is not None:
        loss_dict['data'] = loss_data.item()
    
    return total_loss, loss_dict


In [ ]:
# ============================================================================
# STEP 1: Prepare Experimental Data (if available)
# ============================================================================
# If you have experimental data, load it here and format it as tensors.
# The data should contain: x, z, t (coordinates) and c (concentration measurements)

# Example: Load experimental data from a file or create synthetic data
# Replace this with your actual data loading code
use_experimental_data = True  # Set to True when you have experimental data

if use_experimental_data:
    # Option 1: Load from CSV file
    import pandas as pd
    data = pd.read_csv('experimental_data.csv')
    x_exp = torch.tensor(data['x'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    z_exp = torch.tensor(data['z'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    t_exp = torch.tensor(data['t'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    c_exp = torch.tensor(data['c'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    n_exp = x_exp.shape[0]
    # # Option 2: Create synthetic data for testing (replace with real data)
    # n_exp = 100  # Number of experimental data points
    # x_exp = torch.rand(n_exp, 1, device=device) * (x_max - x_min) + x_min
    # z_exp = torch.rand(n_exp, 1, device=device) * (z_max - z_min) + z_min
    # t_exp = torch.rand(n_exp, 1, device=device) * (t_max - t_min) + t_min
    # # For testing, generate synthetic measurements (replace with actual measurements)
    # c_exp = torch.rand(n_exp, 1, device=device) * 0.3 + 0.35  # Random values between 0.35 and 0.65
    
    exp_data = {
        'x': x_exp,
        'z': z_exp,
        't': t_exp,
        'c': c_exp
    }
    print(f"Loaded {n_exp} experimental data points")
else:
    exp_data = None
    print("No experimental data provided. Lambda will be learned from physics constraints only.")

# ============================================================================
# STEP 2: Training Setup
# ============================================================================
# Note: model.parameters() now includes the learnable Lambda parameter
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1000)

# Loss weights (can be adjusted)
# Increase 'data' weight if you have experimental data to emphasize data fitting
loss_weights = {
    'pde': 1.0,
    'bc_inlet': 10.0,
    'bc_top': 10.0,
    'bc_bottom': 10.0,
    'ic': 10.0,
    'data': 50.0 if use_experimental_data else 0.0  # Higher weight for experimental data
}

# Training parameters
n_epochs = 20000
print_interval = 500
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs) # option 2: cosine annealing

# Training loop
loss_history = []
start_time = time.time()

print("Starting training...")
if use_experimental_data:
    print(f"{'Epoch':<10} {'Total Loss':<15} {'PDE':<15} {'BC Inlet':<15} {'BC Top':<15} {'BC Bottom':<15} {'IC':<15} {'Lambda':<15} {'Data':<15}")
    print("-" * 120)
else:
    print(f"{'Epoch':<10} {'Total Loss':<15} {'PDE':<15} {'BC Inlet':<15} {'BC Top':<15} {'BC Bottom':<15} {'IC':<15} {'Lambda':<15}")
    print("-" * 120)

try:
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        # Compute loss
        total_loss, loss_dict = compute_loss(model, train_data, Pe, k, loss_weights, exp_data=exp_data)
        
        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
#         scheduler.step(total_loss)
        scheduler.step()
        
        # Store loss history
        loss_history.append({
            'epoch': epoch,
            'total': total_loss.item(),
            **loss_dict
        })
        
        # Print progress
        if (epoch + 1) % print_interval == 0 or epoch == 0:
            if use_experimental_data:
                print(f"{epoch+1:<10} {total_loss.item():<15.6e} {loss_dict['pde']:<15.6e} "
                      f"{loss_dict['bc_inlet']:<15.6e} {loss_dict['bc_top']:<15.6e} "
                      f"{loss_dict['bc_bottom']:<15.6e} {loss_dict['ic']:<15.6e} "
                      f"{loss_dict.get('lambda', float('nan')):<15.6f} {loss_dict.get('data', float('nan')):<15.6f}")
            else:
                print(f"{epoch+1:<10} {total_loss.item():<15.6e} {loss_dict['pde']:<15.6e} "
                      f"{loss_dict['bc_inlet']:<15.6e} {loss_dict['bc_top']:<15.6e} "
                      f"{loss_dict['bc_bottom']:<15.6e} {loss_dict['ic']:<15.6e} "
                      f"{loss_dict.get('lambda', float('nan')):<15.6f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user.")
except Exception as e:
    print(f"\nError during training: {e}")
    import traceback
    traceback.print_exc()

elapsed_time = time.time() - start_time
print(f"\nTraining completed in {elapsed_time:.2f} seconds")


In [ ]:
# Plot loss history
if len(loss_history) == 0:
    print("Warning: loss_history is empty. Please run the training cell first.")
else:
    if use_experimental_data:
        loss_history_array = np.array([(h['epoch'], h['total'], h['pde'], h['bc_inlet'], 
                                    h['bc_top'], h['bc_bottom'], h['ic'], h['data']) for h in loss_history])
    else:
        loss_history_array = np.array([(h['epoch'], h['total'], h['pde'], h['bc_inlet'], 
                                    h['bc_top'], h['bc_bottom'], h['ic']) for h in loss_history])
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 8))
    
    # Total loss
    axes[0].semilogy(loss_history_array[:, 0], loss_history_array[:, 1], 'b-', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Total Loss', fontsize=12)
    axes[0].set_title('Total Loss History', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    if 'n_epochs' in globals():
        axes[0].set_xlim([0, n_epochs])
    else:
        axes[0].set_xlim([0, loss_history_array[-1, 0] + 1])
    
    # Individual losses
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 2], 'r-', label='PDE', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 3], 'g-', label='BC Inlet', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 4], 'm-', label='BC Top', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 5], 'c-', label='BC Bottom', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 6], 'y-', label='IC', linewidth=2)
    if use_experimental_data:
        axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 7], 'k-', label='Data', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Individual Loss Components', fontsize=14)
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    if 'n_epochs' in globals():
        axes[1].set_xlim([0, n_epochs])
    else:
        axes[1].set_xlim([0, loss_history_array[-1, 0] + 1])
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize solution at final time
model.eval()
with torch.no_grad():
    # Create grid for visualization
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Evaluate at final time
    t_final = torch.ones_like(X_vis.reshape(-1, 1)) * t_max
    c_final = model(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_final)
    C_final = c_final.reshape(n_x, n_z).cpu().numpy()
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    levels = np.linspace(0.0, 1.0, 51)
    im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), 1 - C_final, levels=levels, cmap='hot')
    ax.set_xlabel(r'$\tilde{x}$', fontsize=14)
    ax.set_ylabel(r'$\tilde{z}$', fontsize=14)
    ax.set_title(f'Concentration $c_s$ at $\\tilde{{t}}={t_max}$ (Λ={model.Lambda.item():.2f}, Pe={Pe})', fontsize=14)
    cbar = plt.colorbar(im, ax=ax, label='Concentration $c_s$')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize solution evolution over time
model.eval()
with torch.no_grad():
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Plot at different times
    times = [0.0, 2.5, 5.0, 7.5, 10.0]
    levels = np.linspace(0.0, 1.0, 51)
    fig, axes = plt.subplots(1, len(times), figsize=(20, 4))
    
    for idx, t_val in enumerate(times):
        t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_val
        c_vis = model(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
        C_vis = c_vis.reshape(n_x, n_z).cpu().numpy()
        
        im = axes[idx].contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), 1-C_vis, levels=levels, cmap='hot')
        im.set_clim(0.0, 1.0)  # Explicitly set color limits
        axes[idx].set_xlabel(r'$\tilde{x}$', fontsize=12)
        axes[idx].set_ylabel(r'$\tilde{z}$', fontsize=12)
        axes[idx].set_title(f'$\\tilde{{t}}={t_val}$', fontsize=12)
        axes[idx].set_aspect('equal')
        cbar = plt.colorbar(im, ax=axes[idx])
        # cbar.set_clim(0.0, 1.0)  # Explicitly set colorbar limits
    
    plt.suptitle(f'Concentration Evolution (Λ={model.Lambda.item():.2f}, Pe={Pe})', fontsize=14)
    plt.tight_layout()
    plt.show()


In [ ]:
# Generate movie of solution evolution over time
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib

model.eval()
with torch.no_grad():
    # Create grid for visualization
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Time points for animation
    n_frames = 100
    time_points = np.linspace(t_min, t_max, n_frames)
    
    # Pre-compute all frames
    print("Pre-computing frames for animation...")
    frames = []
    for i, t_val in enumerate(time_points):
        t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_val
        c_vis = model(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
        C_vis = c_vis.reshape(n_x, n_z).cpu().numpy()
        frames.append(1 - C_vis)  # Convert to c_s (small particle concentration)
        if (i + 1) % 20 == 0:
            print(f"  Processed {i + 1}/{n_frames} frames")
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))
    levels = np.linspace(0.0, 1.0, 51)
    
    # Initial frame
    im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), frames[0], 
                     levels=levels, cmap='hot')
    im.set_clim(0.0, 1.0)
    ax.set_xlabel(r'$\tilde{x}$', fontsize=14)
    ax.set_ylabel(r'$\tilde{z}$', fontsize=14)
    ax.set_aspect('equal')
    cbar = plt.colorbar(im, ax=ax, label='Concentration $c_s$')
    title = ax.set_title(f'Concentration Evolution: $\\tilde{{t}}={time_points[0]:.2f}$ (Λ={model.Lambda.item():.2f}, Pe={Pe})', 
                         fontsize=14)
    
    # Animation update function
    def animate(frame_idx):
        # Clear only the contour plot and title, keep axes and colorbar
        for collection in ax.collections:
            collection.remove()
        ax.set_title(f'Concentration Evolution: $\\tilde{{t}}={time_points[frame_idx]:.2f}$ (Λ={model.Lambda.item():.2f}, Pe={Pe})', 
                     fontsize=14)
        # Create new contour plot
        im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), frames[frame_idx], 
                         levels=levels, cmap='hot')
        im.set_clim(0.0, 1.0)
        return im.collections
    
    # Create animation
    print("Creating animation...")
    anim = FuncAnimation(fig, animate, frames=n_frames, interval=100, blit=False, repeat=True)
    
    # Save as movie file
    print("Saving movie...")
    try:
        # Try using FFMpegWriter (requires ffmpeg to be installed)
        writer = FFMpegWriter(fps=10, metadata=dict(artist='PINN'), bitrate=1800)
        anim.save('solution_evolution_phase2.1.mp4', writer=writer)
        print("Movie saved as 'solution_evolution_phase2.1.mp4'")
    except Exception as e:
        print(f"Error saving with FFMpegWriter: {e}")
        print("Trying alternative method with PillowWriter...")
        try:
            from matplotlib.animation import PillowWriter
            writer = PillowWriter(fps=10)
            anim.save('solution_evolution_phase2.1.gif', writer=writer)
            print("Movie saved as 'solution_evolution_phase2.1.gif'")
        except Exception as e2:
            print(f"Error saving with PillowWriter: {e2}")
            print("Displaying animation inline instead...")
            plt.show()
    
    # Also display the animation inline
    plt.tight_layout()
    plt.show()


In [ ]:
# Check PDE residual at final time
model.eval()
# Sample points in the domain
n_check = 1000
x_check = torch.rand(n_check, 1, device=device) * (x_max - x_min) + x_min
z_check = torch.rand(n_check, 1, device=device) * (z_max - z_min) + z_min
t_check = torch.ones(n_check, 1, device=device) * t_max

residual, c_check = compute_pde_residual(x_check, z_check, t_check, model, Pe, k)

# Detach for statistics (no need for gradients)
residual_detached = residual.detach()

print(f"PDE Residual Statistics at t={t_max}:")
print(f"  Mean: {torch.mean(torch.abs(residual_detached)).item():.6e}")
print(f"  Std:  {torch.std(residual_detached).item():.6e}")
print(f"  Max:  {torch.max(torch.abs(residual_detached)).item():.6e}")
print(f"  Min:  {torch.min(torch.abs(residual_detached)).item():.6e}")

# Plot residual distribution
fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(residual_detached.cpu().numpy().flatten(), bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('PDE Residual', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of PDE Residuals at Final Time', fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Plot Lambda evolution during training
if len(loss_history) == 0:
    print("Warning: loss_history is empty. Please run the training cell first.")
else:
    lambda_values = [h['lambda'] for h in loss_history]
    epochs = [h['epoch'] for h in loss_history]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(epochs, lambda_values, 'b-', linewidth=2)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Lambda', fontsize=12)
    ax.set_title('Lambda Evolution During Training', fontsize=14)
    ax.grid(True, alpha=0.3)
    if 'n_epochs' in globals():
        ax.set_xlim([0, n_epochs])
    else:
        ax.set_xlim([0, epochs[-1] + 1])
    plt.tight_layout()
    plt.show()

In [ ]:
# Compute and plot residuals between PINN solution and experimental data
if exp_data is None:
    print("No experimental data available. Cannot compute residuals.")
else:
    model.eval()
    with torch.no_grad():
        # Get experimental data
        x_exp = exp_data['x']
        z_exp = exp_data['z']
        t_exp = exp_data['t']
        c_exp = exp_data['c']  # Actual experimental measurements
        
        # Predict concentrations at experimental points
        c_pred = model(x_exp, z_exp, t_exp)
        
        # Compute residuals (predicted - actual)
        residuals = (c_pred - c_exp).cpu().numpy().flatten()
        c_pred_np = c_pred.cpu().numpy().flatten()
        c_exp_np = c_exp.cpu().numpy().flatten()
        
        # Statistics
        mean_residual = np.mean(residuals)
        std_residual = np.std(residuals)
        mse = np.mean(residuals**2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(residuals))
        
        print("Residual Statistics:")
        print(f"  Mean residual: {mean_residual:.6e}")
        print(f"  Std residual:  {std_residual:.6e}")
        print(f"  MSE:           {mse:.6e}")
        print(f"  RMSE:          {rmse:.6e}")
        print(f"  MAE:           {mae:.6e}")
        
        # Create comprehensive residual plots
        fig = plt.figure(figsize=(16, 12))
        
        # 1. Histogram of residuals
        ax1 = plt.subplot(2, 3, 1)
        ax1.hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
        ax1.axvline(0, color='r', linestyle='--', linewidth=2, label='Zero residual')
        ax1.axvline(mean_residual, color='g', linestyle='--', linewidth=2, label=f'Mean: {mean_residual:.4e}')
        ax1.set_xlabel('Residual (Predicted - Actual)', fontsize=12)
        ax1.set_ylabel('Frequency', fontsize=12)
        ax1.set_title('Distribution of Residuals', fontsize=14)
        ax1.legend(fontsize=10)
        ax1.grid(True, alpha=0.3)
        
        # 2. Scatter plot: Predicted vs Actual
        ax2 = plt.subplot(2, 3, 2)
        ax2.scatter(c_exp_np, c_pred_np, alpha=0.6, s=20, color='steelblue')
        # Perfect prediction line (y=x)
        min_val = min(c_exp_np.min(), c_pred_np.min())
        max_val = max(c_exp_np.max(), c_pred_np.max())
        ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect prediction')
        ax2.set_xlabel('Actual Concentration', fontsize=12)
        ax2.set_ylabel('Predicted Concentration', fontsize=12)
        ax2.set_title('Predicted vs Actual Concentration', fontsize=14)
        ax2.legend(fontsize=10)
        ax2.grid(True, alpha=0.3)
        ax2.set_aspect('equal', adjustable='box')
        
        # 3. Residual vs Predicted (to check for heteroscedasticity)
        ax3 = plt.subplot(2, 3, 3)
        ax3.scatter(c_pred_np, residuals, alpha=0.6, s=20, color='steelblue')
        ax3.axhline(0, color='r', linestyle='--', linewidth=2)
        ax3.set_xlabel('Predicted Concentration', fontsize=12)
        ax3.set_ylabel('Residual', fontsize=12)
        ax3.set_title('Residual vs Predicted', fontsize=14)
        ax3.grid(True, alpha=0.3)
        
        # 4. Residual vs x coordinate
        ax4 = plt.subplot(2, 3, 4)
        x_exp_np = x_exp.cpu().numpy().flatten()
        ax4.scatter(x_exp_np, residuals, alpha=0.6, s=20, color='steelblue')
        ax4.axhline(0, color='r', linestyle='--', linewidth=2)
        ax4.set_xlabel(r'$\tilde{x}$', fontsize=12)
        ax4.set_ylabel('Residual', fontsize=12)
        ax4.set_title('Residual vs x-coordinate', fontsize=14)
        ax4.grid(True, alpha=0.3)
        
        # 5. Residual vs z coordinate
        ax5 = plt.subplot(2, 3, 5)
        z_exp_np = z_exp.cpu().numpy().flatten()
        ax5.scatter(z_exp_np, residuals, alpha=0.6, s=20, color='steelblue')
        ax5.axhline(0, color='r', linestyle='--', linewidth=2)
        ax5.set_xlabel(r'$\tilde{z}$', fontsize=12)
        ax5.set_ylabel('Residual', fontsize=12)
        ax5.set_title('Residual vs z-coordinate', fontsize=14)
        ax5.grid(True, alpha=0.3)
        
        # 6. Residual vs time
        ax6 = plt.subplot(2, 3, 6)
        t_exp_np = t_exp.cpu().numpy().flatten()
        ax6.scatter(t_exp_np, residuals, alpha=0.6, s=20, color='steelblue')
        ax6.axhline(0, color='r', linestyle='--', linewidth=2)
        ax6.set_xlabel(r'$\tilde{t}$', fontsize=12)
        ax6.set_ylabel('Residual', fontsize=12)
        ax6.set_title('Residual vs Time', fontsize=14)
        ax6.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Additional: Q-Q plot for residual normality check
        from scipy import stats
        fig, ax = plt.subplots(figsize=(8, 6))
        stats.probplot(residuals, dist="norm", plot=ax)
        ax.set_title('Q-Q Plot of Residuals (Normality Check)', fontsize=14)
        ax.set_xlabel('Theoretical Quantiles', fontsize=12)
        ax.set_ylabel('Sample Quantiles', fontsize=12)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Save the model
file_name = './models/pinn_inverse_Lambda.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'Lambda': model.Lambda.item(),  # Save the learned Lambda value
    'Lambda_init': Lambda_init,     # Save initial Lambda for reference
    'Pe': Pe,
    'k': k,
    'tEnd': tEnd,
    'loss_history': loss_history,
    'layers': layers,  # Save network architecture
    'x_min': x_min, 'x_max': x_max,
    'z_min': z_min, 'z_max': z_max,
    't_min': t_min, 't_max': t_max,
    'pde points': n_pde,
    'bc_inlet points': n_bc_inlet,
    'bc_top points': n_bc_top,
    'bc_bottom points': n_bc_bottom,
    'bc_outlet points': n_bc_outlet,
    'ic points': n_ic,
    'exp points': n_exp
}, file_name)
print(f"Model saved to '{file_name}'")


In [ ]:
# Optional: Load a saved model
# checkpoint = torch.load('pinn_segregation_model.pth')
# 
# # Restore model architecture and state
# if 'layers' in checkpoint:
#     layers = checkpoint['layers']
#     model = PINN(layers, lambda_init=checkpoint.get('Lambda_init', checkpoint['Lambda'])).to(device)
# else:
#     # If layers not saved, use default architecture
#     layers = [3, 50, 50, 50, 50, 1]
#     model = PINN(layers, lambda_init=checkpoint.get('Lambda_init', checkpoint['Lambda'])).to(device)
# 
# model.load_state_dict(checkpoint['model_state_dict'])
# 
# # Restore parameters
# Lambda_init = checkpoint.get('Lambda_init', checkpoint['Lambda'])  # Use Lambda_init if available, else use Lambda
# Pe = checkpoint['Pe']
# k = checkpoint['k']
# tEnd = checkpoint['tEnd']
# loss_history = checkpoint['loss_history']
# 
# # Update domain bounds based on tEnd
# t_max = tEnd
# 
# print("Model loaded successfully!")
# print(f"  Learned Lambda: {model.Lambda.item():.6f}")
# print(f"  Initial Lambda: {Lambda_init:.6f}")
# print(f"  Pe: {Pe}, k: {k}, tEnd: {tEnd}")
